# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from generative_social_choice.utils.helper_functions import get_base_dir_path

In [3]:
from generative_social_choice.slates.voting_algorithms import (
    SequentialPhragmenMinimax,
    GreedyTotalUtilityMaximization,
    ExactTotalUtilityMaximization,
    LPTotalUtilityMaximization,
    VotingAlgorithm,
    GeometricTransformation,
)
from generative_social_choice.slates.voting_algorithm_axioms import (
    IndividualParetoAxiom,
    HappiestParetoAxiom,
    CoverageAxiom,
    MinimumAndTotalUtilityParetoAxiom,
    VotingAlgorithmAxiom,
    NonRadicalMinUtilityAxiom,
    NonRadicalTotalUtilityAxiom,
)
from generative_social_choice.test.utilities_for_testing import rated_vote_cases

# Load Data

In [4]:
voting_algorithm_evals_dir = get_base_dir_path() / "data" / "voting_algorithm_evals"
latest = True
if latest:
    file = sorted(voting_algorithm_evals_dir.glob("*.csv"))[-1]
else:
    file = voting_algorithm_evals_dir / "2025-01-20-180945.csv"
file

df = pd.read_csv(file, index_col=0, header=[0, 1])

In [5]:
file

WindowsPath('G:/Other computers/My Computer/NTFS/dev/generative_social_choice/generative_social_choice/data/voting_algorithm_evals/2025-10-25-010655.csv')

In [6]:
# Select subset of rows
# df = df.iloc[-8:]
df

vote                                                       Simple 1  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=None)              1.0   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=Ge...              1.0   
Phragmen(marginal_slate, clear=True, redist=False)              1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...              1.0   
Phragmen(marginal_previous, clear=True, redist=...              1.0   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=None)                                    1.0   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                                    1.0   
Phragmen(marginal_slate, clear=True, redist=False)                                    1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...                                    1.0   
Phragmen(marginal_previous, clear=True, redist=...                                    1.0   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=None)                          1.0   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                          1.0   
Phragmen(marginal_slate, clear=True, redist=False)                          1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...                          1.0   
Phragmen(marginal_previous, clear=True, redist=...                          1.0   

vote                                                                                                    \
subtest                                            Minimum Utility and Total Utility Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                                1.0    
ExactTotalUtilityMaximization(utility_transform...                                                1.0    
LPTotalUtilityMaximization(utility_transform=None)                                                1.0    
GreedyTotalUtilityMaximization(utility_transfor...                                                1.0    
ExactTotalUtilityMaximization(utility_transform...                                                1.0    
LPTotalUtilityMaximization(utility_transform=Ge...                                                1.0    
Phragmen(marginal_slate, clear=True, redist=False)                                                1.0    
Phragmen(marginal_slate, clear=False, redist=Fa...                                                1.0    
Phragmen(marginal_previous, clear=True, redist=...                                                1.0    

vote                                                                                              \
subtest      

## Overall Performance

In [7]:
df.sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                              150.625000
ExactTotalUtilityMaximization(utility_transform=None)                               159.458333
LPTotalUtilityMaximization(utility_transform=None)                                  154.833333
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))    151.083333
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))     161.708333
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))        154.250000
Phragmen(marginal_slate, clear=True, redist=False)                                  143.458333
Phragmen(marginal_slate, clear=False, redist=False)                                 143.916667
Phragmen(marginal_previous, clear=True, redist=True)                                144.083333
dtype: float64

In [8]:
pd.DataFrame(np.trunc(df.values + 0/24+1*1e-6), index=df.index, columns=df.columns).sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                              121.0
ExactTotalUtilityMaximization(utility_transform=None)                               157.0
LPTotalUtilityMaximization(utility_transform=None)                                  150.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))    132.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))     159.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))        152.0
Phragmen(marginal_slate, clear=True, redist=False)                                  118.0
Phragmen(marginal_slate, clear=False, redist=False)                                 111.0
Phragmen(marginal_previous, clear=True, redist=True)                                113.0
dtype: float64

## Differential Test Performance

In [9]:
algs_for_non_uniform_rows = df.loc[[ExactTotalUtilityMaximization().name, ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)).name], :]

non_uniform_columns = algs_for_non_uniform_rows.loc[:, algs_for_non_uniform_rows.nunique() > 1]
non_uniform_columns


vote                                                                           Ex 1.1 modified  \
subtest                                            Non-radical Total Utility Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                    0.416667   
ExactTotalUtilityMaximization(utility_transform...                                    1.000000   

vote                                                                                            Ex 3.1  \
subtest                                            Minimum Utility and Total Utility Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                              1.000    
ExactTotalUtilityMaximization(utility_transform...                                              0.875    

vote                                                                                              \
subtest                                            Non-radical Minimum Utility Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                      1.000000   
ExactTotalUtilityMaximization(utility_transform...                                      0.958333   

vote                                                                                            \
subtest                                            Non-radical Total Utility Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                       1.000   
ExactTotalUtilityMaximization(utility_transform...                                       0.875   

vote                                                                                    Ex 4.1  \
subtest                                            Non-radical Total Utility Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                         0.5   
ExactTotalUtilityMaximization(utility_transform...                                         1.0   

vote                                                                                    Ex B.3  \
subtest                                            Non-radical Total Utility Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                    0.666667   
ExactTotalUtilityMaximization(utility_transform...                                    1.000000   

vote                                                                                    Ex C.2  \
subtest                                            Non-radical Total Utility Pareto Efficiency   
ExactTotalUtilityMaximization(utility_transform...                                    0.291667   
ExactTotalUtilityMaximization(utility_transform...                                    1.000000   

vote                                                                                 Ex Alg2.1  
subtest                                            Non-radical Total Utility Pareto Efficiency  
ExactTotalUtilityMaximization(utility_transform...                                    0.583333  
ExactTotalUtilityMaximization(utility_transform...                                    1.000000

These counts show which test cases and which axioms most frequently show unique behavior across the `VotingAlgorithm`s

In [10]:
# Count the number of columns with each name in both levels of the MultiIndex
level_0_counts = non_uniform_columns.columns.get_level_values(0).value_counts()
level_1_counts = non_uniform_columns.columns.get_level_values(1).value_counts()

print("Counts for level 0:")
print(level_0_counts)
print("\nCounts for level 1:")
print(level_1_counts)



Counts for level 0:
vote
Ex 3.1             3
Ex 1.1 modified    1
Ex 4.1             1
Ex B.3             1
Ex C.2             1
Ex Alg2.1          1
Name: count, dtype: int64

Counts for level 1:
subtest
Non-radical Total Utility Pareto Efficiency            6
Minimum Utility and Total Utility Pareto Efficiency    1
Non-radical Minimum Utility Pareto Efficiency          1
Name: count, dtype: int64


## Axiom Pass Rate

In [11]:
df.groupby(df.columns.get_level_values(1), axis=1).mean()

C:\Users\aaron\AppData\Local\Temp\ipykernel_5340\1810388850.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df.groupby(df.columns.get_level_values(1), axis=1).mean()


subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Minimum Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),0.998457,0.919753,0.912037,0.908951,0.848765,0.990741
ExactTotalUtilityMaximization(utility_transform=None),1.000000,1.000000,1.000000,1.000000,0.905864,1.000000
LPTotalUtilityMaximization(utility_transform=None),1.000000,0.962963,0.961420,0.959877,0.884259,0.966049
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,0.913580,0.912037,0.904321,0.881173,0.984568
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,1.000000,0.995370,0.998457,0.995370,1.000000
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,0.939815,0.941358,0.945988,0.942901,0.942901
"Phragmen(marginal_slate, clear=True, redist=False)",0.955247,0.879630,0.871914,0.820988,0.854938,0.930556
"Phragmen(marginal_slate, clear=False, redist=False)",0.956790,0.890432,0.887346,0.796296,0.858025,0.941358
"Phragmen(marginal_previous, clear=True, redist=True)",0.966049,0.891975,0.873457,0.799383,0.876543,0.929012


# Debuggign

In [12]:
# alg = GreedyTotalUtilityMaximization()
# alg = LPTotalUtilityMaximization()
# alg = SequentialPhragmenMinimax()
# alg = ExactTotalUtilityMaximization()
alg = ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = SequentialPhragmenMinimax(load_magnitude_method="marginal_slate", clear_reassigned_loads=False, redistribute_defected_candidate_loads=False)

In [13]:
df1 = df.loc[alg.name,:].unstack()
df1

subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Minimum Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
vote,,,,,,
Ex 1.1,1.0,1.0,1.000,1.000000,1.000,1.0
Ex 1.1 modified,1.0,1.0,1.000,1.000000,1.000,1.0
Ex 1.2,1.0,1.0,1.000,1.000000,1.000,1.0
Ex 1.3,1.0,1.0,1.000,1.000000,1.000,1.0
Ex 2.1,1.0,1.0,1.000,1.000000,1.000,1.0
Ex 2.2,1.0,1.0,1.000,1.000000,1.000,1.0
Ex 3.1,1.0,1.0,0.875,0.958333,0.875,1.0
Ex 4.1,1.0,1.0,1.000,1.000000,1.000,1.0
Ex 4.2,1.0,1.0,1.000,1.000000,1.000,1.0


In [ ]:
# case = rated_vote_cases["Ex 1.1 modified"]
from generative_social_choice.slates.voting_algorithms import RatedVoteCase
from generative_social_choice.slates.voting_utils import (
    CellWiseAugmentation, CandidateWiseAugmentation, VoterAndCellWiseAugmentation)


# case = rated_vote_cases["Ex A.1"]
case = rated_vote_cases["Ex 3.1"]

case = RatedVoteCase(
    name="Ex 3.1_BIG",
    rated_votes=case.rated_votes,
    slate_size=case.slate_size,
    noise_augmentation_methods = {
                CellWiseAugmentation(): 40, 
                CandidateWiseAugmentation(): 100, 
                VoterAndCellWiseAugmentation(): 100
            }
)
# case = rated_vote_cases["Ex 4.3"]
# case = rated_vote_cases["Simple 3"]
axiom = CoverageAxiom()
# axiom = HappiestParetoAxiom()
# axiom = IndividualParetoAxiom()
# axiom = MinimumAndTotalUtilityParetoAxiom()
# axiom = NonRadicalTotalUtilityAxiom()
# axiom = NonRadicalMinUtilityAxiom()

case.rated_votes

,s1,s2,s3,s4,s5,s6
0,2,0,1,0,0,0
1,1,2,0,0,0,0
2,0,1,2,0,0,0
3,0,0,0,2,0,1
4,0,0,0,1,2,0
5,0,0,0,0,1,2


In [29]:
alg.name

'ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))'

In [30]:
# df.loc[:,df.columns.get_level_values(1) == axiom.name].iloc[:,10:-10]
df.loc[alg.name,df.columns.get_level_values(1) == axiom.name]
# df.columns.get_level_values(1) == type(axiom).__name__
# df.columns.get_level_values(1), axiom.name


vote             subtest         
Simple 1         Maximum Coverage    1.0
Simple 2         Maximum Coverage    1.0
Simple 3         Maximum Coverage    1.0
Ex 1.1           Maximum Coverage    1.0
Ex 1.1 modified  Maximum Coverage    1.0
Ex 1.2           Maximum Coverage    1.0
Ex A.1           Maximum Coverage    1.0
Ex 1.3           Maximum Coverage    1.0
Ex 2.1           Maximum Coverage    1.0
Ex 2.2           Maximum Coverage    1.0
Ex 3.1           Maximum Coverage    1.0
Ex 4.1           Maximum Coverage    1.0
Ex 4.2           Maximum Coverage    1.0
Ex 4.3           Maximum Coverage    1.0
Ex 4.4           Maximum Coverage    1.0
Ex B.1           Maximum Coverage    1.0
Ex B.2           Maximum Coverage    1.0
Ex B.3           Maximum Coverage    1.0
Ex C.1           Maximum Coverage    1.0
Ex C.2           Maximum Coverage    1.0
Ex D.1           Maximum Coverage    1.0
Ex Alg1.3        Maximum Coverage    1.0
Ex Alg1.4        Maximum Coverage    1.0
Ex Alg1.5        Maximu

In [31]:
alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)[0]

['s1', 's2', 's5']

In [ ]:
# axiom.satisfactory_slates(rated_votes=case.rated_votes, slate_size=case.slate_size)
# case.augmented_cases[0]

,s1,s2,s3,s4,s5,s6
0,2.077481,0.040411,1.093472,0.082362,0.095575,0.082106
1,1.032542,2.070424,0.027544,0.093560,0.012012,0.063964
2,0.093994,1.037862,2.011768,0.032117,0.010914,0.096776
3,0.007910,0.042695,0.078443,2.047317,0.066054,1.092586
4,0.005460,0.080445,0.049045,1.061849,2.027595,0.078439
5,0.070658,0.081048,0.046021,0.085551,1.083640,2.049228


In [33]:
num_aug_cases = len(case.augmented_cases)
aug_case_votes = pd.DataFrame(index = range(num_aug_cases), columns=["rated_votes", "assignments", "axiom_slates", "alg_slate", "satisfied"])
for i, aug_case in enumerate(case.augmented_cases):
    # print(aug_case)
    aug_case_votes.at[i, "rated_votes"] = aug_case
    aug_case_votes.at[i, "axiom_slates"] = axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "alg_slate"], aug_case_votes.at[i, "assignments"] = alg.vote(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "satisfied"] = axiom.evaluate_assignment(
        rated_votes=aug_case,
        assignments=aug_case_votes.at[i, "assignments"],
        slate_size=case.slate_size,
    )
    # print(axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size))
# aug_case_votes.iloc[:,2:]

In [34]:
failures = aug_case_votes[aug_case_votes["satisfied"] == False]
failures.iloc[:,2:]


,axiom_slates,alg_slate,satisfied
24,"[(s2, s4, s5), (s2, s4, s6), (s2, s5, s6)]","[s1, s2, s5]",False
30,"[(s1, s3, s6)]","[s2, s4, s6]",False
158,"[(s1, s3, s6), (s2, s5, s6)]","[s3, s5, s6]",False
164,"[(s1, s2, s4), (s1, s3, s4)]","[s1, s3, s5]",False
167,"[(s1, s2, s4), (s1, s2, s6), (s1, s3, s4), (s3...","[s1, s2, s5]",False
196,"[(s1, s2, s4), (s1, s2, s5), (s1, s3, s4), (s1...","[s1, s2, s6]",False
238,"[(s3, s4, s6)]","[s3, s5, s6]",False


In [35]:
f0 = failures.iloc[0]
f0.rated_votes
# (f0.rated_votes - case.rated_votes)*1e6


,s1,s2,s3,s4,s5,s6
0,1.956304,-0.007640,0.945486,-0.039251,-0.015138,-0.033798
1,0.907156,1.911891,-0.089469,-0.094976,-0.017050,-0.011134
2,-0.016823,0.984120,1.920018,-0.093614,-0.040049,-0.015578
3,-0.013109,-0.064670,-0.038733,1.940263,-0.083165,0.944636
4,-0.088376,-0.058317,-0.011817,0.955010,1.955756,-0.022364
5,-0.070032,-0.038849,-0.071768,-0.099232,0.984806,1.997223


In [36]:
((f0.rated_votes - case.rated_votes)*1e6)

,s1,s2,s3,s4,s5,s6
0,-43695.639228,-7639.782203,-54514.115809,-39251.384639,-15138.258410,-33797.754367
1,-92844.158214,-88108.955663,-89468.520624,-94975.604158,-17049.840648,-11134.130106
2,-16823.253489,-15880.401807,-79982.058461,-93614.080139,-40048.871799,-15578.212869
3,-13109.158549,-64670.389494,-38732.800317,-59737.175285,-83165.144057,-55364.491978
4,-88375.942086,-58317.457430,-11817.363018,-44990.456654,-44244.261706,-22363.589524
5,-70031.916788,-38848.744995,-71768.425284,-99232.475053,-15194.139236,-2777.271107


In [37]:
f0.assignments

,candidate_id
0,s1
1,s2
2,s2
3,s1
4,s5
5,s5


In [38]:
alg.vote(rated_votes=f0.rated_votes, slate_size=case.slate_size)

(['s1', 's2', 's5'],
   candidate_id
 0           s1
 1           s2
 2           s2
 3           s1
 4           s5
 5           s5)

In [41]:
axiom.evaluate_assignment(
    rated_votes=f0.rated_votes,
    assignments=f0.assignments,
    slate_size=case.slate_size,
)

False

In [40]:
from kiwiutils.kiwilib import leafClasses
[cls.__name__ for cls in leafClasses(VotingAlgorithmAxiom)]

['NonRadicalTotalUtilityAxiom',
 'CoverageAxiom',
 'HappiestParetoAxiom',
 'NonRadicalMinUtilityAxiom',
 'IndividualParetoAxiom',
 'MinimumAndTotalUtilityParetoAxiom']

# Scratch